In [5]:
from pinecone_code import *
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain 
from langchain_community.chat_models import ChatCohere
from langchain.chains.conversation.memory import  ConversationBufferWindowMemory, ConversationSummaryBufferMemory                                                
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.callbacks import get_openai_callback
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import dotenv
dotenv.load_dotenv()

# PENDING : Move these to a config file
INDEX_NAME = 'langchain-retrieval-transcript'
PINECONE_VECTOR_DIMENSION = 3072 #1536
PINECONE_UPSERT_BATCH_LIMIT = 90
PINECONE_TOP_K_RESULTS = 3
DELTA = 5
CLOUD_PROVIDER = 'aws'
REGION = 'us-west-2'
METRIC = 'cosine'

EMBEDDING = 'OpenAI'
EMBEDDING_MODEL = 'text-embedding-3-large' #'text-embedding-ada-002'

NAMESPACE = 'default_namespace'
master_json_file = 'master_meeting_details'

LLM_MODEL = 'gpt-3.5-turbo'
LLM_TEMPERATURE = 0.0
CONV_BUFFER_MEMORY_WINDOW = 1
LLM_SUMMARY_MAX_TOKEN_LIMIT = 650
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [20]:
class LangChain:
    def __init__(self):
        self.pinecone_obj = PineconeServerless()
        #self.llm=ChatOpenAI(temperature=LLM_TEMPERATURE, model_name=LLM_MODEL, streaming=False)
        self.llm=ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3)
        # replace self.llm with UR LLM
        
        #self.llm=ChatCohere(model='command', temperature=0)
        #self.conversation_bufw = ConversationChain(llm=self.llm, memory=ConversationBufferWindowMemory(k=CONV_BUFFER_MEMORY_WINDOW))
        self.conversation_bufw = ConversationChain(llm=self.llm, memory=ConversationSummaryBufferMemory(llm=self.llm,max_token_limit=LLM_SUMMARY_MAX_TOKEN_LIMIT))
    def prompt(self, query, context):
        system_template = SystemMessagePromptTemplate.from_template(
            'You are a helpful assistant.'
            'You are provided with a context below. You are expected to answer the user query based on the context below.'
            'The context provided is a part of transcript of a meeting, in the format:'
            'Conversations in meeting: <meeting_title>'
            'Start Time - Speaker: Text \n'

            'You will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.'
            'You answers should be concise and relevant to the context.'
            'You can mention the meeting_title in your response if you want to refer to the meeting.'
            'You are not allowed to talk about anything else other than the context below.'
            'You cannot use any external information other than the context below.'
            'No need to greet or say goodbye. Just answer the user query based on the context below.'
            'You can also skip mentioning phrases such as : Based on the context provided. Instead simply answer the user query based on the context below.\n\n'
            'Context:\n'
            '{context}'
        )
        # system_template = SystemMessagePromptTemplate.from_template(
        #     'You are a helpful assistant.'
        #     'You will answer the user query based on the context below.'
        #     'You are also provided with the chat history of the user query and the response. You can use this information to answer the user query as well'
        #     'Context: \n'
        #     '{context}'
        # )

        human_template = HumanMessagePromptTemplate.from_template(' \nUser Query: {input}')
        chat_prompt = ChatPromptTemplate.from_messages([system_template, human_template])
        
        chat_prompt_value = chat_prompt.format_prompt(
            context = context,
            input = query
        )
        print(chat_prompt_value)
        return chat_prompt_value.to_messages()


    def query_chatbot(self, query, context):
        self.messages = self.prompt(query, context)
        resp = self.conversation_bufw(self.messages)
        #resp = self.count_tokens(self.conversation_bufw, self.messages)
        print(resp)
        #self.messages.append(resp)
        # append just the query and response
        
        return resp
        #return resp['response']
    
    def parse_conversations(self, conversations) -> str:
        data = []
        for cluster_id, cluster_df in conversations.items():
            with open(f'bin/data/default_namespace/{cluster_id}.json') as f:
                meeting_data = json.load(f)
                meeting_title = meeting_data['meeting_title']
                data.append(f"Conversations in meeting '{meeting_title}':")
                for i, row in cluster_df.iterrows():
                    data.append(f"{row['start_time']} - {row['speaker']}: {row['text']}")
                data.append("\n\n")
        data = '\n'.join(data)
        return data

    def chat(self, query, in_filter: list[str]=[], complete_db_flag:bool = True):
        if 'summary' in query:
            pass
        self.pinecone_obj.query_pinecone(query, in_filter, complete_db_flag)
        conversation = self.pinecone_obj.query_delta_conversations()
        #print(conversation)
        context = self.parse_conversations(conversation)
        #print(context)
        response = self.query_chatbot(query, context)
        return response
    
    def count_tokens(self, chain, query):
        with get_openai_callback() as callback:
            response = chain(query)
            print(f'Call Back:  {callback}')
            return response
        
    

In [21]:
obj = LangChain()

In [22]:
obj = LangChain()
#obj.chat("How much is the compensation for the job?")
obj.chat("Do people need a reading room?")

pinecone query response
{'matches': [{'id': '10705',
              'metadata': {'meeting_uuid': '52d105f8-1c80-4056-8253-732b9e2bec63',
                           'speaker': 'spk_0',
                           'start_time': '00:05:16',
                           'text': 'And do we need a room? Like a reading '
                                   'room?'},
              'score': 0.822736561,
              'values': []},
             {'id': '10889',
              'metadata': {'meeting_uuid': '9ed1fefa-db53-41fc-a21b-479b67e30073',
                           'speaker': 'spk_2',
                           'start_time': '00:04:02',
                           'text': 'And what about the reading room?'},
              'score': 0.735501528,
              'values': []},
             {'id': '11051',
              'metadata': {'meeting_uuid': '9ed1fefa-db53-41fc-a21b-479b67e30073',
                           'speaker': 'spk_3',
                           'start_time': '00:25:37',
                 

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


{'input': [SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet or say goodbye. Just answer the user query based on the context below.You can also skip mentioning phrases such as : Based on the context provided. Instead simply answer the user query based on the context below.\n\nContext:\nConversations in mee

{'input': [SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet or say goodbye. Just answer the user query based on the context below.You can also skip mentioning phrases such as : Based on the context provided. Instead simply answer the user query based on the context below.\n\nContext:\nConversations in mee

In [ ]:
queries = [
    "What is a common recommendation regarding the age limit for children on social media platforms?",
    "What was discussed about cyberbullying?",
    "Did they talk about social media and eating disorder?",
    "Does social media make people antisocial?",
    "Who is a pro social media user?",
    "What was discussed about global warming?"
]

ground_truths = [
    "Children under the age of 13 should not be allowed on social media.",
    "Cyberbullying experience shared, including instances from a dance team.",
    "Mention of real advice on eating disorders and depression found on social media.",
    "Discussion on social media's potential to foster both social and antisocial behavior.",
    "Alex, a 16-year-old, shares positive experiences with social media as a trans individual.",
    "Discussion on the reality and causality of global warming, supported by scientific evidence."
]


In [23]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np
# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.
✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [ ]:
from trulens_eval import TruChain
tru_rag = TruChain(obj.conversation_bufw,
    chain_id = 'chain_Gemini',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [ ]:
query = "How much is the compensation for the job?"
context = "The job would pay around 32000$"
obj.query_chatbot(query, context)

In [ ]:
query = "What's the job location"
context = "Job location is in New York"
obj.query_chatbot(query, context)

In [ ]:
query = "Whom should I contact for the job?"
context = "You can contact the HR department for the job"
obj.query_chatbot(query, context)

In [ ]:
query = "What's the job location?"
context = ""
obj.query_chatbot(query, context)

In [ ]:
query = "How much is the compensation for the job?"
context = ""
obj.query_chatbot(query, context)

In [ ]:
query = "Did you get the answer to the last question from the chat history?"
context = " "
obj.query_chatbot(query, context)

In [ ]:
query = "What's the job location"
context = ""
obj.query_chatbot(query, context)

In [5]:
obj.chat("Whats the minimum age that you need to be in order to do shadowing")

Fetch window:  ['34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']
Fetch window:  ['46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56']
Fetch window:  ['40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']
messages=[SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet o

{'input': [SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet or say goodbye. Just answer the user query based on the context below.You can also skip mentioning phrases such as : Based on the context provided. Instead simply answer the user query based on the context below.\n\nContext:\nConversations in mee

In [6]:
obj.chat("Who was talking about electives?")

Fetch window:  ['56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66']
Fetch window:  ['53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63']
Fetch window:  ['51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61']
messages=[SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet o

{'input': [SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet or say goodbye. Just answer the user query based on the context below.You can also skip mentioning phrases such as : Based on the context provided. Instead simply answer the user query based on the context below.\n\nContext:\nConversations in mee

In [7]:
obj.chat("Why were the volunteer programs canceled?")

Fetch window:  ['68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78']
Fetch window:  ['70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80']
Fetch window:  ['72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82']
messages=[SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet o

{'input': [SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet or say goodbye. Just answer the user query based on the context below.You can also skip mentioning phrases such as : Based on the context provided. Instead simply answer the user query based on the context below.\n\nContext:\nConversations in mee

In [8]:
obj.chat("The minimum age to do what is 17?")

Fetch window:  ['35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45']
Fetch window:  ['34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']
Fetch window:  ['76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86']
messages=[SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet o

{'input': [SystemMessage(content="You are a helpful assistant.You are provided with a context below. You are expected to answer the user query based on the context below.The context provided is a part of transcript of a meeting, in the format:Conversations in meeting: <meeting_title>Start Time - Speaker: Text \nYou will respond using the context below only. If you cannot find an answer from the below context, you can ask for more information.You answers should be concise and relevant to the context.You can mention the meeting_title in your response if you want to refer to the meeting.You are not allowed to talk about anything else other than the context below.You cannot use any external information other than the context below.No need to greet or say goodbye. Just answer the user query based on the context below.You can also skip mentioning phrases such as : Based on the context provided. Instead simply answer the user query based on the context below.\n\nContext:\nConversations in mee